In [21]:
import pandas as pd

nombre_csv = 4160
dp = pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')
# Filtrar por una fecha específica, por ejemplo '2024-11-01'
fecha_deseada = '2024-11-21'
datos_filtrados_dp = dp[dp['Fecha'] >= fecha_deseada]

# Mostrar los datos filtrados
datos_filtrados_dp[['Fecha','Hora','nivel_estres']]

,Fecha,Hora,nivel_estres
2472,2024-11-21,0,normal
2473,2024-11-21,1,normal
2474,2024-11-21,2,normal
2475,2024-11-21,3,normal
2476,2024-11-21,4,normal
2477,2024-11-21,5,normal
2478,2024-11-21,6,normal
2479,2024-11-21,7,normal
2480,2024-11-21,8,normal
2481,2024-11-21,9,normal


In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Parámetros
nombre_csv = 4160
timesteps = 24

# Cargar el modelo guardado
model_cargado = load_model(f'entrenamientos/{nombre_csv}modelo_entrenado.h5')

# Cargar los datos
df =pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')

# Filtrar por una fecha específica
fecha_deseada = '2024-11-21'
df_filtrado = df[df['Fecha'] >= fecha_deseada]

# Seleccionar características para predicción
X_test = df_filtrado[['period eating', 'period other', 'period resting', 'period rumination']]

# Estandarizar las características
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Validar que hay suficientes datos para crear secuencias
if len(X_test_scaled) > timesteps:
    # Crear secuencias
    X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
    print("Forma de X_test_reshaped:", X_test_reshaped.shape)
    
    # Predecir las etiquetas
    y_pred_proba = model_cargado.predict(X_test_reshaped)
    y_pred_classes = np.argmax(y_pred_proba, axis=1)
    
    # Resultados
    print("Predicciones de probabilidad:", y_pred_proba)
    print("Clases predichas:", y_pred_classes)
else:
    print("No hay suficientes datos para generar secuencias de 24 timesteps.")


Forma de X_test_reshaped: (24, 24, 4)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
Predicciones de probabilidad: [[9.9681193e-01 3.1875246e-03 6.3923790e-07]
 [9.9954283e-01 4.5506004e-04 2.1345018e-06]
 [9.9992907e-01 7.0887458e-05 5.3051277e-08]
 [9.9994433e-01 5.4018717e-05 1.6481811e-06]
 [9.9984694e-01 1.5249682e-04 6.3822654e-07]
 [9.9999261e-01 7.3497422e-06 4.9965390e-08]
 [9.9999976e-01 2.1041663e-07 1.3190404e-08]
 [1.0000000e+00 4.0892675e-10 2.3170481e-11]
 [1.0000000e+00 8.1436229e-18 5.0207717e-13]
 [1.0000000e+00 4.4733241e-21 2.9221230e-17]
 [1.0000000e+00 1.6771496e-11 2.1215677e-15]
 [1.0000000e+00 6.1607615e-13 1.0727472e-19]
 [1.0000000e+00 1.1045736e-10 5.5886433e-17]
 [1.0000000e+00 6.6925693e-10 2.2610005e-15]
 [9.9999964e-01 4.0214343e-07 5.9454285e-16]
 [9.9994588e-01 5.4133601e-05 7.7174629e-14]
 [9.9998009e-01 1.9879666e-05 7.9791547e-12]
 [1.0000000e+00 2.4044899e-09 1.0572884e-12]
 [1.0000000e+00 1.1962628e-09 3.6885578e-10]
 [1.0000000e+00 3.7630430e-11 2.992714

In [20]:
X_test_reshaped

array([[[-0.15787144, -0.39781497, -0.08482843,  0.51757313],
        [-0.93121496, -0.39781497, -0.21955594,  1.7935034 ],
        [-0.93121496, -0.39781497,  0.83408227,  0.55807885],
        ...,
        [-0.93121496, -0.39781497,  0.86862779,  0.51757313],
        [-0.93121496, -0.39781497, -0.56846565,  2.20261121],
        [ 0.28644956, -0.39781497,  0.54044539, -0.85152031]],

       [[-0.93121496, -0.39781497, -0.21955594,  1.7935034 ],
        [-0.93121496, -0.39781497,  0.83408227,  0.55807885],
        [-0.93121496, -0.39781497,  1.17262832,  0.16112276],
        ...,
        [-0.93121496, -0.39781497, -0.56846565,  2.20261121],
        [ 0.28644956, -0.39781497,  0.54044539, -0.85152031],
        [-0.93121496, -0.39781497, -0.49246551,  2.11349862]],

       [[-0.93121496, -0.39781497,  0.83408227,  0.55807885],
        [-0.93121496, -0.39781497,  1.17262832,  0.16112276],
        [-0.93121496, -0.39781497,  1.27626487,  0.0396056 ],
        ...,
        [ 0.28644956, -0.39